<html>
<head>

</head>
<body style="backgroud-color:blue;">
<h2>PyTorch Basics</h2><br> 
Author:  <b>Ayushman Gupta</b>
</body>
</html>

In [1]:
#importing library
import torch
import torch.nn as nn
import torchvision
import torch.utils.data as data_utils
import numpy as np
from PIL import Image as P
import torchvision.transforms as transforms
import pandas as pd
import torchvision.datasets as dset
from torch.utils.data import Dataset
from IPython.core.display import Image, display
import matplotlib.pyplot as plt

### Creating Tensors

In [ ]:
x = torch.tensor(1, requires_grad=True)
w = torch.tensor(2, requires_grad=True)
b = torch.tensor(3, requires_grad=True)


In [ ]:
# Computational Graph
y = w*x + b

In [ ]:
# Compute Gradients
y.backward()

In [ ]:
# Printing Gradients
print(x.grad)
print(w.grad)
print(b.grad)

### autograd 

In [ ]:
x = torch.rand(10,3)
y = torch.rand(10,2)

### Build FullyConnected Layer

In [ ]:
# default bias= True
linear = nn.Linear(3,2)

In [ ]:
display(Image('linearNode.JPG', width=300, unconfined=True))

In [ ]:
linear

In [ ]:
print("weight: ",linear.weight)

In [ ]:
print("bias: ",linear.bias)

### Build loss and Optimizer

In [ ]:
criterion = nn.MSELoss()
optimizer = torch.optim.SGD(params=linear.parameters(),lr=.01)

In [ ]:
pred = linear(x)

In [ ]:
loss = criterion(pred,y)

In [ ]:

print("Loss: {0:.4}".format(loss.item()))

In [ ]:
loss.backward()


In [ ]:
#printing the gradient

print("dL/dW \n",linear.weight.grad)
print("dL/db \n",linear.bias.grad)

In [ ]:
# Updates the gradient
optimizer.step()

In [ ]:
pred = linear(x)
loss = criterion(pred,y)
print("Loss after 1 iteration {0:.4}".format(loss.item()))

In [ ]:
# loss after 100 step
for i in range(10):
    
    loss.backward()
    optimizer.step()
    pred = linear(x)
    loss = criterion(pred,y)
    print("Loss after n iteration {0:.4}".format(loss.item()))

### Loading data from Numpy Array

In [ ]:
x = np.array([[1,2],[3,4]])

In [ ]:
y = torch.from_numpy(x)

In [ ]:
y.numpy() 

### Create Custom Dataset

In [ ]:
df = pd.DataFrame(columns=['image','label'])


In [ ]:
df.loc[0] = ['img/c1.jpg',0]
df.loc[1] = ['img/c2.jpg',1]

In [ ]:
df.to_csv('cat.csv',header=True)

In [ ]:
transform = transforms.Compose([
       
    transforms.Resize((64,64)),
    transforms.ToTensor()
])
trainSet = torchvision.datasets.ImageFolder('img',transform=transform)
trainloader = torch.utils.data.DataLoader(trainSet,batch_size=4,shuffle=True)
classes = ('cats','dogs')

In [ ]:
data_iter  = iter(trainloader)

In [ ]:
images,labels = data_iter.next()

In [ ]:
image

In [ ]:
for image,label in trainloader:
    print(image,label)

In [ ]:
trainloader.batch_sampler.sampler.data_source.samples

In [ ]:
#plot batch images

def imshow(inp_tensor, title=None):
    """image show for Tensor"""
    img = inp_tensor.numpy().transpose((1, 2, 0))
    plt.imshow(img)
    if title is not None:
        plt.title(title)
    plt.pause(0.001)

### Plotting Images in a batch

In [ ]:


images,labels = data_iter.next()
batch_imgs = torchvision.utils.make_grid(images)
imshow(batch_imgs, title=[classes[x] for x in labels])

## Pretrained Model

In [7]:
# Download and Load the pretrained Model
resnet18 = torchvision.models.resnet18(pretrained=True)

In [8]:
for param in resnet18.parameters():
    param.requires_grad =False

In [9]:
resnet18.fc  = nn.Linear(resnet18.fc.in_features,100)

In [10]:
# forward Pass

images = torch.rand(64,3,224,224)
output = resnet18(images)

In [11]:
print(len(output[0]))
print(output.size())


100
torch.Size([64, 100])


### Save a Model 
#### (Entire Model)

In [ ]:
torch.save(resnet18,'resnet18n.ckpt')

In [3]:
model = torch.load('resnet18n.ckpt')

In [6]:
model(images)

tensor([[-2.4458e-01,  6.8148e-01,  5.9854e-01,  ...,  4.9777e-01,
          8.0814e-01,  1.9875e-01],
        [-4.4622e-01, -1.1292e-01, -3.5059e-01,  ...,  3.9052e-01,
          9.9688e-01, -1.1106e-01],
        [-8.5354e-01,  4.3576e-01,  1.2909e-01,  ...,  4.3072e-01,
          1.1038e+00, -8.3854e-02],
        ...,
        [-4.0239e-01, -1.6502e-03,  4.5889e-01,  ...,  2.7348e-01,
         -8.3825e-02,  2.4314e-01],
        [-5.4329e-01, -8.8589e-02,  1.3915e-01,  ...,  4.7511e-01,
          6.2503e-01,  2.5680e-01],
        [-4.9356e-01,  2.3851e-01,  3.8922e-01,  ...,  2.4511e-01,
          1.1542e+00,  1.6226e-01]])

### Save a Model 
#### (Only the model parameters

In [12]:
torch.save(resnet18.state_dict(),'param.ckpt')

In [13]:
resnet18.load_state_dict(torch.load('param.ckpt'))